## Day 7: The Sum of Its Parts

https://adventofcode.com/2018/day/7

### Part 1

Put the alphabetically first step that has no dependencies at the start of the sequence. Repeat until finished. Let `networkx` do the hefty lifting. (This needs version 2+ of `networkx`.)

In [1]:
from parse import parse
import networkx as nx


# instructions are tuples (x, y) where x must be
# sequenced before y
def parse_instructions(strings):
    return [parse('Step {} must be finished before step {} can begin.', s).fixed 
            for s in strings]


def sequence_steps(instructions):
    dependencies = nx.DiGraph()
    dependencies.add_edges_from(instructions)
    
    sequence = ''
    
    while dependencies:
        next_step = min([n for n, d in dependencies.in_degree if d == 0])
        sequence += next_step
        dependencies.remove_node(next_step)
    
    return sequence

In [2]:
test_input = '''Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.'''.splitlines()

test_instructions = parse_instructions(test_input)
test_instructions

[('C', 'A'),
 ('C', 'F'),
 ('A', 'B'),
 ('A', 'D'),
 ('B', 'E'),
 ('D', 'E'),
 ('F', 'E')]

In [3]:
sequence_steps(test_instructions)

'CABDFE'

In [4]:
instructions = parse_instructions(open('input', 'r'))

sequence_steps(instructions)

'BCADPVTJFZNRWXHEKSQLUYGMIO'